In [ ]:
import requests
from bs4 import BeautifulSoup
import bs4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import pandas as pd
service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

Функция get_pages принимает на вход страницы на которых расположен выбор машин. На одной странице около 37 машин. Функция собирает ссылки на все машины на этой странице

In [ ]:
def get_pages(url):
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html')
    dirty_hrefs = soup.find_all('a', attrs={'class': 'Link ListingItemTitle__link'})
    clean_hrefs = [a['href'] for a in dirty_hrefs]
    return clean_hrefs

Дальше мы входим на auto.ru вне циклов, чтобы пройти капчу и забыть про нее

In [ ]:
url = "https://auto.ru/moskva/cars/vendor-chinese/used/"
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html')

Функция base_char работает на первичной странице машины. Она собирает самые важные данные: название, цену, километраж и год

In [ ]:
def base_char(url):
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html')
    # название
    car_name = soup.find('h1', attrs={"class": "CardHead__title"})
    if car_name:
        car_name = car_name.text.split(',')[0]
    # цена
    price = soup.find('span', {'class': 'OfferPriceCaption__price'})
    if price:
        price = price.text.replace(u'\xa0', '')
    # километраж
    km_age = soup.find('li', attrs={'class': "CardInfoRow CardInfoRow_kmAge"})
    if km_age:
        km_age = km_age.find_all('span', attrs={'class': 'CardInfoRow__cell'})
        km_age = int(km_age[1].text.split()[0])
    # год
    year = soup.find('li', attrs={'class': "CardInfoRow CardInfoRow_year"})
    if year:
        year = year.find_all('span', attrs={'class': 'CardInfoRow__cell'})
        year = int(year[1].text.split()[0])
    d = {"Название": car_name,
         "Ссылка": url,
         "Цена": price,
         "Год": year,
         "Километраж": km_age}
    return d


Функиця button_link работает на странице машины. Она собирает ссылки на подробные характеристики автомобиля

In [ ]:
def button_link(url):
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html')
    har = soup.find('a', attrs={"class": "Button Button_color_gray Button_size_l Button_type_link Button_width_full"})
    if har:
        har = har.get('href')
    return har

Функция get_features работает на странице характеристик, ссылку на которую она получает через функцию button_link. Данная функиця собирает большое количество характеристик: класс автомобиля, количество дверей, количество мест, длину, ширину, высоту, коробку передач, тип привода, максимальную скорость, разгон до 100км\ч, тип двигаетля и максимальуню мощность

In [ ]:
def get_features(href):
    driver.get(href)
    soup = BeautifulSoup(driver.page_source, 'html')
    d = {}
    names = [el.text for el in soup.find_all('h3', {'class': 'ModificationInfo__groupName-axo8d'})]
    for j in range(len(names)):
        
        #Общая информация
        if names[j] == 'Общая информация':
            all_info = soup.find_all('ul', {'class': 'ModificationInfo__options-yDaxw'})[j].text.split('\xa0')
            for i in range(0, len(all_info), 2):
                if all_info[i] == 'Класс автомобиля':
                    car_class = all_info[i + 1]
                elif all_info[i] == 'Количество дверей':
                    doors_num = all_info[i + 1]
                elif all_info[i] == 'Количество мест':
                    seats_num = all_info[i + 1]
            
            if 'Класс автомобиля' not in all_info:
                car_class = None
            if 'Количество дверей' not in all_info:
                doors_num = None
            if 'Количество мест' not in all_info:
                seats_num = None
            
        #Размеры
        elif names[j] == 'Размеры':
            sizes = soup.find_all('ul', {'class': 'ModificationInfo__options-yDaxw'})[j].text.split('\xa0')
            for i in range(0, len(sizes), 2):
                if sizes[i] == 'Длина':
                    car_length = sizes[i + 1]
                elif sizes[i] == 'ммШирина':
                    car_width = sizes[i + 1]
                elif sizes[i] == 'ммВысота':
                    car_height = sizes[i + 1]
    
            if 'Длина' not in sizes:
                car_length = None
            if 'ммШирина' not in sizes:
                car_width = None
            if 'ммВысота' not in sizes:
                car_height = None
        
    # трансмиссия
        elif names[j] == 'Трансмиссия':
            transmission = soup.find_all('ul', {'class': 'ModificationInfo__options-yDaxw'})[j].text.split('\xa0')
            for i in range(0, len(transmission), 2):
                if transmission[i] == 'Коробка передач':
                    korobka = transmission[i + 1]
                elif transmission[i] == 'Тип привода':
                    privod = transmission[i + 1]
    
            if 'Коробка передач' not in transmission:
                korobka = None
            if 'Тип привода' not in transmission:
                privod = None
        
    # Эксплуатационные показатели
        elif names[j] == 'Эксплуатационные показатели':
            expl = soup.find_all('ul', {'class': 'ModificationInfo__options-yDaxw'})[j].text.split('\xa0')
            for i in range(0, len(expl), 2):
                if expl[i] == 'Максимальная скорость':
                    max_speed = expl[i + 1]
                elif expl[i] == 'км/чРазгон до 100 км/ч':
                    time_100 = expl[i + 1]
                
            if 'Максимальная скорость' not in expl:
                max_speed = None
            if 'км/чРазгон до 100 км/ч' not in expl:
                time_100 = None
            
    # Двигатель
        elif names[j] == 'Двигатель':
            engine = soup.find_all('ul', {'class': 'ModificationInfo__options-yDaxw'})[j].text.split('\xa0')
            for i in range(0, len(engine), 2):
                if engine[i] == 'Тип двигателя':
                    engine_type = engine[i + 1]
                elif engine[i] == 'Максимальная мощность':
                    engine_hp = engine[i + 1]
            
            if 'Тип двигателя' not in engine:
                engine_type = None
            if 'Максимальная мощность' not in engine:
                engine_type = None
            
    d = {'Класс автомобиля': car_class, 'Количество дверей': doors_num, 'Количество мест': seats_num,
         'Длина': car_length, 'Ширина': car_width, 'Высота': car_height,
         'Коробка передач': korobka, 'Тип привода':  privod, 'Максимальная скорость': max_speed,
         'Разгон до 100': time_100, 'Тип двигателя': engine_type, 'Максимальная мощность': engine_hp}
    
    return d

Дальше написан сам парсер который использует вышеописанные функции. Он идет по 6 городам России: 3 в центральной части и 3 в восточной. Сначала парсер испольлзует функцию по сбору страниц, далее проходит по каждой машине со страницы на которой он собирает базовые и дополнительные характеристики автомобиля. Также в парсере записывается информация о городе где расположена машина.

In [ ]:
cities = ['moskva', 'rostov-na-donu', 'sankt-peterburg', 'ekaterinburg', 'novosibirsk', 'vladivostok']
parsed_df = []
for city in cities:
    res = []
    for i in range(2, 15):
        url = f'https://auto.ru/{city}/cars/vendor-chinese/used/?page={i}'
        page_pages = get_pages(url)
        for page in page_pages:
            car_1 = base_char(page)
            car_1["Город"] = city
            inside_url = button_link(page)
            if inside_url:
                car_2 = get_features(inside_url)
                car = car_1 | car_2
            else:
                car = car_1
            parsed_df.append(car)


Затем полученные список словарей преобразовывается в датафрейм с помощью pandas

In [ ]:
df = pd.DataFrame(parsed_df)
df.to_csv('df', encoding='utf-8')